In [12]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [2]:
# 분류 대상 카테고리 선택하기 
categories = ["top","bottom","etc"]
nb_classes = len(categories)

In [3]:
# 이미지 크기 지정 
image_w = 64 
image_h = 64
pixels = image_w * image_h * 3

In [13]:
# 이미지 데이터 읽어 들이기 
X = []
Y = []

for idx, cat in enumerate(categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 이미지 
    image_dir = "./clothes_test/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    for i, f in tqdm_notebook(enumerate(files)):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
        # if i % 10 == 0:
        #     print(i, "\n", data[:10])


<ipython-input-13-a8b5309a97f2>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, f in tqdm_notebook(enumerate(files)):


C:\Users\user\anaconda3\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



In [14]:
X = np.array(X)
Y = np.array(Y)

In [15]:
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

xy = (X_train, X_test, y_train, y_test)

In [16]:
print(len(X_train), len(X_test))
print(X_train.shape, X_test.shape)

3095 774
(3095, 64, 64, 3) (774, 64, 64, 3)


In [17]:
print('>>> data 저장중 ...')
np.save("./clothes.npy", xy, allow_pickle=True)
print("ok,", len(Y))

>>> data 저장중 ...
C:\Users\user\anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
ok, 3869


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

In [32]:
# 카테고리 지정하기
categories = ["top", 'bottom', 'etc']
nb_classes = len(categories)

In [20]:
# 이미지 크기 지정하기
image_w = 64
image_h = 64

In [21]:
# 데이터 열기 
X_train, X_test, y_train, y_test = np.load("./clothes.npy", allow_pickle=True)

In [22]:
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float")  / 255
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (3095, 64, 64, 3)
X_test shape: (774, 64, 64, 3)


In [23]:
# 모델 구조 정의 
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [24]:
# 전결합층
model.add(Flatten())    # 벡터형태로 reshape
model.add(Dense(512))   # 출력
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [25]:
# 모델 구축하기
model.compile(loss='categorical_crossentropy',   # 최적화 함수 지정
    optimizer='rmsprop',
    metrics=['accuracy'])
# 모델 확인
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 128)       7

In [26]:
# 학습 완료된 모델 저장
hdf5_file = "./clothes.h5"
# 파일로 저장
model.fit(X_train, y_train, epochs=10, batch_size=10)
model.save_weights(hdf5_file)

Epoch 1/10
310/310 [==============================] - 92s 292ms/step - loss: 1.0000 - accuracy: 0.6078
Epoch 2/10
310/310 [==============================] - 89s 286ms/step - loss: 0.4962 - accuracy: 0.7964
Epoch 3/10
310/310 [==============================] - 89s 287ms/step - loss: 0.4127 - accuracy: 0.8441
Epoch 4/10
310/310 [==============================] - 90s 289ms/step - loss: 0.3344 - accuracy: 0.8757
Epoch 5/10
310/310 [==============================] - 87s 280ms/step - loss: 0.2750 - accuracy: 0.8965
Epoch 6/10
310/310 [==============================] - 86s 276ms/step - loss: 0.2516 - accuracy: 0.9137
Epoch 7/10
310/310 [==============================] - 85s 273ms/step - loss: 0.2136 - accuracy: 0.9285
Epoch 8/10
310/310 [==============================] - 85s 275ms/step - loss: 0.1795 - accuracy: 0.9350
Epoch 9/10
310/310 [==============================] - 85s 273ms/step - loss: 0.1581 - accuracy: 0.9471
Epoch 10/10
310/310 [==============================] - 84s 271ms/step - l

In [27]:
# 모델 평가하기 
score = model.evaluate(X_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

25/25 [==============================] - 3s 101ms/step - loss: 0.6319 - accuracy: 0.8966
loss= 0.6318702697753906
accuracy= 0.8966408371925354


In [38]:
## 이미지 넣고 적중률 시험
# 적용해볼 이미지 
test_image = './test3.jpg'

In [39]:
# 이미지 resize
img = Image.open(test_image)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)

In [40]:
X = np.array(data)
X = X.astype("float") / 255
X = X.reshape(-1, 64, 64,3)

In [41]:
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])

New data category :  etc
